# Notebook to perform the vertical transformation from z-levels to sigma coordinates for Temperature and Salinity

The vertical transformation is performed using linear interpolation similarly as in model2roms

In [99]:
# Import packages
import numpy as np
import xarray as xr

from tqdm import tqdm

In [2]:
# import data

# Import the data that needs to be vertically transformed (MODEL grid)
data_in = xr.open_dataset('horizontally_regridded.nc')

# Import ROMS grid
grid = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/data/grid/NorthSea4_smooth01_sponge_nudg.nc')

data = data_in.drop(['zos'])



### Create reanalysis data and ROMS input grids

In [3]:
def create_grdMODEL(data):
    ''' Function to create the dataset grdMODEL '''
    
    grdMODEL = data.copy().drop(['thetao', 'so', 'uo', 'vo'])
    
    grdMODEL['lon'] = data.lon
    grdMODEL['lat'] = data.lat
    grdMODEL['h'] = data.depth
    grdMODEL['nlevels'] = grdMODEL.h.size
    
    
    grdMODEL['fillval'] = -32767   # Change for ORA-20C
    grdMODEL['hc'] = None

    # Create grid for ESMF interpolation, probably not needed for VT

    grdMODEL['z_r'] = -grdMODEL.h

    grdMODEL['grdType'] = 'regular'
    grdMODEL['lonName'] = 'longitude'
    grdMODEL['latName'] = 'latitude'
    grdMODEL['depthName'] = 'depth'


    grdMODEL['Lp'] = len(grdMODEL.lat[1,:])
    grdMODEL['Mp'] = len(grdMODEL.lat[:,1])

    grdMODEL['L'] = grdMODEL.Lp - 1
    grdMODEL['M'] = grdMODEL.Mp - 1
    
    
    
    
    
    return grdMODEL


grdMODEL = create_grdMODEL(data)


data = data.drop(['lat', 'lon'])

In [4]:
def create_grdROMS(grid):
    ''' Function to create the dataset grdROMS '''
    
    # Create the dataset grdROMS

    # Copy the roms grid
    grdROMS = grid.copy()

    # Drop unnecessary variables
    grdROMS = grdROMS.drop(['tracer_NudgeCoef', 'diff_factor', 'visc_factor', 'hraw', 'f', 'spherical'])



    # Add below variables to grdROMS
    grdROMS['write_clim'] = True
    grdROMS['write_bry'] = True
    grdROMS['write_init'] = True
    grdROMS['write_stations'] = False
    grdROMS['lonname'] = 'lon_rho'
    grdROMS['latname'] = 'lat_rho'
    grdROMS['inittime'] = 0                    # Set initTime to 1 if you dont want the first time-step to be the initial field (no ubar and vbar if time=0)
    grdROMS['ocean_time'] = 0
    grdROMS['NT'] = 2
    grdROMS['tracer'] = grdROMS.NT
    grdROMS['time'] = 0                      
    grdROMS['reftime'] = 0
    grdROMS['grdtype'] = 'regular'

    grdROMS['masked_h'] = grdROMS.h.where(grdROMS.h > 0, grdROMS.h, grdROMS.h.max())
    grdROMS['hmin'] = grdROMS.masked_h.min()

    grdROMS['vtransform'] = 2
    grdROMS['vstretching'] = 4

    grdROMS['nlevels'] = grdROMS.s_rho.size

    grdROMS['zeta'] = (('eta_rho', 'xi_rho'), np.zeros(grdROMS.h.shape))

    grdROMS['invpm'] = 1.0 / grdROMS.pm
    grdROMS['invpn'] = 1.0 / grdROMS.pn

    grdROMS['Lp'] = grdROMS.lat_rho[1,:].size     
    grdROMS['Mp'] = grdROMS.lat_rho[:,1].size     

    grdROMS['fillval'] = -9.99e33

    grdROMS['eta_rho_'] = grdROMS.Mp
    grdROMS['eta_u_'] = grdROMS.Mp
    grdROMS['eta_v_'] = grdROMS.Mp - 1
    grdROMS['eta_psi_'] = grdROMS.Mp - 1


    grdROMS['xi_rho_'] = grdROMS.Lp
    grdROMS['xi_u_'] = grdROMS.Lp - 1
    grdROMS['xi_v_'] = grdROMS.Lp
    grdROMS['xi_psi_'] = grdROMS.Lp - 1



    # Obtain s_rho

    c1 = 1.0
    c2 = 2.0
    p5 = 0.5

    lev = np.arange(1, int(grdROMS.nlevels) + 1, 1)
    ds = 1.0 / int(grdROMS.nlevels)


    grdROMS['s_rho_'] = - c1 + (lev - p5) * ds


    # Obtain s_w

    lev = np.arange(0, int(grdROMS.nlevels), 1)
    ds = 1.0 / (int(grdROMS.nlevels) - 1)


    grdROMS['s_w_'] = - c1 + (lev - p5) * ds




    # Obtain Cs_r

    if (grdROMS.theta_s > 0):
        Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_rho)) / (np.cosh(grdROMS.theta_s) - c1)

    else:
        Csur = -grdROMS.s_rho**2

    if (grdROMS.theta_b > 0):
        Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
        grdROMS['Cs_r'] = Cbot
    else:
        grdROMS['Cs_r'] = Csur     



    # Obtain Cs_w

    if (grdROMS.theta_s > 0):
        Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_w)) / (np.cosh(grdROMS.theta_s) - c1)

    else:
        Csur = -grdROMS.s_w**2

    if (grdROMS.theta_b > 0):
        Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
        grdROMS['Cs_w'] = Cbot
    else:
        grdROMS['Cs_w'] = Csur     




    # Obtain z_r

    z0 = (grdROMS.hc * grdROMS.s_rho + grdROMS.h * grdROMS.Cs_r) / (grdROMS.hc + grdROMS.h)
    grdROMS['z_r'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0



    # Obtain z_w

    z0 = (grdROMS.hc * grdROMS.s_w + grdROMS.h * grdROMS.Cs_w) / (grdROMS.hc + grdROMS.h)
    grdROMS['z_w'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0



    # Also ESMF grid is added but probably not needed for VT



    grdROMS['L'] = grdROMS.Lp -1
    grdROMS['M'] = grdROMS.Mp -1

    
    
    
    
    return grdROMS


grdROMS = create_grdROMS(grid)


### Check input data for fill-values and extremes

In [5]:
def is_clean(dataset):
    return bool((dataset != grdMODEL['fillval']).all().all())


print(is_clean(data.thetao))
print(is_clean(data.so))
print(is_clean(data.vo))
print(is_clean(data.uo))


True
True
True
True


The glorys data variables don't contain any fillval

In [6]:
print(int(data.thetao.max()))
print(int(data.so.max()))
print(int(data.uo.max()))
print(int(data.vo.max()))

24
36
0
0


In [7]:
print(int(data.thetao.min()))
print(int(data.so.min()))
print(int(data.uo.min()))
print(int(data.vo.min()))

0
21
0
0


There seem to be no fill-values and huge extremes in this dataset

### Prepare for vertical transformation

In [8]:
# Indices of output data, however my output data includes Time
index_time = data.time.size


outINDEX_ST = (int(grdROMS.nlevels), index_time, int(grdROMS.eta_rho_), int(grdROMS.xi_rho_))
outINDEX_U = (int(grdROMS.nlevels), int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_UBAR = (int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_V = (int(grdROMS.nlevels), int(grdROMS.eta_v_), int(grdROMS.xi_v_))
outINDEX_VBAR = (int(grdROMS.eta_v_), int(grdROMS.xi_v_))


outdataST = np.empty((outINDEX_ST))
outdataU = np.empty((outINDEX_U))
outdataUBAR = np.empty((outINDEX_UBAR))
outdataV = np.empty((outINDEX_V))
outdataVBAR = np.empty((outINDEX_VBAR))

This slow vertical trans

In [9]:
grdROMS.z_r.min()

<xarray.DataArray 'z_r' ()>
array(-5346.15781958)

In [10]:
grdMODEL.z_r[-1]

<xarray.DataArray 'z_r' ()>
array(-5727.917, dtype=float32)
Coordinates:
    depth    float32 5.728e+03

In [11]:
def vertical_transformation_slow(outdat, dat, grdROMS, grdMODEL, II, JJ):
    ''' Function to perform the vertical transformation. Function takes about 13 hours per variable. '''
    
    # Obtain variables
    bathymetry = grdROMS.h
    zr = grdROMS.z_r
    zs = grdMODEL.z_r
    Nroms = grdROMS.nlevels
    Ndata = grdMODEL.nlevels
    xi_rho = int(grdROMS.xi_rho_)
    eta_rho = int(grdROMS.eta_rho_)
    fill=-10000

    
    # Transpose dimensions
    zr = zr.transpose('s_rho', 'eta_rho', 'xi_rho')
    
    
    # Change the arrangememnt of zr
    zr = zr.sortby('s_rho', ascending = False)
    
    
    
    # Perform vertical transformation
    for jc in tqdm(range(JJ)): # Loop over eta_rho direction (110)

        for ic in range(II): # Loop over xi_rho direction (122)

            for kc in range(int(Nroms)): # Loop over ROMS depth layers (30)

                # Case 2: ROMS depth layer is shallower than GLORYS depth layer. 

                if zr[kc, jc, ic] > zs[0]:    # zs[0] = -0.494025, only the case for kc = 0

                    outdat[kc, :, jc, ic] = dat[0, :, jc, ic]


                else:

                    for kT in range(int(Ndata) - 1): # Do loop between surface and bottom of GLORYS depth layers, - 1 because we also check for the next GLORYS layer each step

                        # Case 3: ROMS depth layer is deeper than some GLORYS depth layer, but shallower than the next GLORYS layer which is below bottom 

                        if (zr[kc, jc, ic] <= zs[kT]) & (-(bathymetry[jc, ic]) > zs[kT+1]):

                            outdat[kc, :, jc, ic] = dat[kT, :, jc, ic]

                            #We do not want to give the deepest depth a fill_value, but there are no fill_values



                        # Case 5: ROMS layer in between two GLORYS layers. This is the typical case for most layers.

                        elif (zr[kc, jc, ic] <= zs[kT]) & (zr[kc, jc, ic] >= zs[kT+1]) & (-bathymetry[jc, ic] <= zs[kT+1]):

                            rz2 = abs((zr[kc, jc, ic] - zs[kT+1]) / (abs(zs[kT+1]) - abs(zs[kT])))

                            rz1 = 1.0-rz2

                            outdat[kc, :, jc, ic] = (rz1 * dat[kT+1, :, jc, ic] + rz2 * dat[kT, :, jc, ic])



    
    return outdat


In [ ]:
outdata_s = vertical_transformation_slow(outdataST, data.so, grdROMS, grdMODEL, int(grdROMS.xi_rho_), int(grdROMS.eta_rho_))

#outdata_s = np.ma.masked_where(abs(outdata_s) > 1000, outdata_s)

In [ ]:
outdata_t = vertical_transformation_slow(outdataST, data.thetao, grdROMS, grdMODEL, int(grdROMS.xi_rho_), int(grdROMS.eta_rho_))

#outdata_t = np.ma.masked_where(abs(outdata_t) > 1000, outdata_t)

In [ ]:
outdata_u = vertical_transformation_slow(outdataU, data.uo, grdROMS, grdMODEL, int(grdROMS.xi_u_), int(grdROMS.eta_u_))

#outdata_u = np.ma.masked_where(abs(outdata_u) > 1000, outdata_u)

In [ ]:
outdata_v = vertical_transformation_slow(outdataV, data.vo, grdROMS, grdMODEL, int(grdROMS.xi_v_), int(grdROMS.eta_v_))

#outdata_v = np.ma.masked_where(abs(outdata_v) > 1000, outdata_v)

In [ ]:
data.thetao

In [ ]:
def obtain_()

In [ ]:
index_time = data.time.size


outINDEX_ST = (int(grdROMS.nlevels), index_time, int(grdROMS.eta_rho_), int(grdROMS.xi_rho_))
outINDEX_U = (int(grdROMS.nlevels), int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_UBAR = (int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_V = (int(grdROMS.nlevels), int(grdROMS.eta_v_), int(grdROMS.xi_v_))
outINDEX_VBAR = (int(grdROMS.eta_v_), int(grdROMS.xi_v_))


outdataST = np.empty((outINDEX_ST))
outdataU = np.empty((outINDEX_U))
outdataUBAR = np.empty((outINDEX_UBAR))
outdataV = np.empty((outINDEX_V))
outdataVBAR = np.empty((outINDEX_VBAR))

In [ ]:
grdROMS.eta_rho

In [ ]:
# Step 1: create output data array using empty numpy arrays of dimensions (s_rho, time, 

In [ ]:
output = xr.DataArray(data = outdataST,
                      dims = ["s_rho", "time", "eta_rho", "xi_rho"],
                      coords = dict(
                          s_rho = grdROMS.s_rho,
                          time = data.time,
                          eta_rho = grdROMS.eta_rho,
                          xi_rho = grdROMS.xi_rho,
                      ),
                      attrs=dict(
                          description="Try out dataarray",
                          units="depends",
                      ),
                     )

In [ ]:
# Step 2: create 1d function to use in apply_ufunc

In [ ]:
# Obtain variables
bathymetry = grdROMS.h
zr = grdROMS.z_r
zs = grdMODEL.z_r
Nroms = grdROMS.nlevels
Ndata = grdMODEL.nlevels
xi_rho = int(grdROMS.xi_rho_)
eta_rho = int(grdROMS.eta_rho_)
fill=-10000

    
# Transpose dimensions
zr = zr.transpose('s_rho', 'eta_rho', 'xi_rho')
    
    
# Change the arrangememnt of zr
zr = zr.sortby('s_rho', ascending = False)
    


def obtain_VT_data(output_data, input_data, zr, zs, Ndata, ):
    ''' Function to obtain the vertical transformed data for a certain depth layer and grid location of the ROMS grid. A time series is returned. '''
    
    
    
    # Create empty array
    outdat = np.empty(input_data.time.size)
    
    
    
    # Case 1: ROMS is deeper than GLORYS. This part searches for deepest good value if ROMS depth is deeper than GLORYS. 
    # This means that if no value, or only fill_value, is available from GLORYS where ROMS is deepest, the closest value from GLORYS is found by looping upward in the water column.

    # Between GLORYS and ROMS, CASE 1 will never happen

    if zr < zs[Ndata - 1]:

        outdat = input_data[Ndata - 1, :, jc, ic]

        #We do not want to give the deepest depth a fill_value, but there are no fill_values



    # Case 2: ROMS depth layer is shallower than GLORYS depth layer. 

    elif zr[kc, jc, ic] > zs[0]:    # zs[0] = -0.494025, only the case for kc = 0

        outdat = input_data[0, :, jc, ic]


    else:

        for kT in range(int(Ndata) - 1): # Do loop between surface and bottom of GLORYS depth layers, - 1 because we also check for the next GLORYS layer each step

            # Case 3: ROMS depth layer is deeper than some GLORYS depth layer, but shallower than the next GLORYS layer which is below bottom 

            if (zr[kc, jc, ic] <= zs[kT]) & (-(bathymetry[jc, ic]) > zs[kT+1]):

                outdat[kc, :, jc, ic] = input_data[kT, :, jc, ic]

                #We do not want to give the deepest depth a fill_value, but there are no fill_values

            # For ORA check whether case 4 is necessary

            # Case 5: ROMS layer in between two GLORYS layers. This is the typical case for most layers.

            elif (zr[kc, jc, ic] <= zs[kT]) & (zr[kc, jc, ic] >= zs[kT+1]) & (-bathymetry[jc, ic] <= zs[kT+1]):

                rz2 = abs((zr[kc, jc, ic] - zs[kT+1]) / (abs(zs[kT+1]) - abs(zs[kT])))

                rz1 = 1.0-rz2

                outdat[kc, :, jc, ic] = (rz1 * input_data[kT+1, :, jc, ic] + rz2 * input_data[kT, :, jc, ic])
                    
                    
                    
    return outdat 
                    
                    

In [ ]:
# Step 3: use apply_ufunc on data array created in step 1

In [ ]:
output_data = xr.apply_ufunc(obtain_VT_data,                                       # The function that should be executed
                             output, data.thetao, grdROMS, grdMODEL,               # The arguments the function needs
                             input_core_dims=[[], [], [], []],                     # For each argument
                             exclude_dims=set(("lat",)),                           # The dimensions that are allowed to change size. Must be set!
                            )

In [ ]:
def vertical_transformation(outdat, dat, grdROMS, grdMODEL, II, JJ):
    ''' Function to perform the vertical transformation. Function takes about 13 hours per variable. '''
    
    # Obtain variables
    bathymetry = grdROMS.h
    zr = grdROMS.z_r
    zs = grdMODEL.z_r
    Nroms = grdROMS.nlevels
    Ndata = grdMODEL.nlevels
    xi_rho = int(grdROMS.xi_rho_)
    eta_rho = int(grdROMS.eta_rho_)
    fill=-10000

    
    # Transpose dimensions
    zr = zr.transpose('s_rho', 'eta_rho', 'xi_rho')
    
    
    # Change the arrangememnt of zr
    zr = zr.sortby('s_rho', ascending = False)
    
    
    
    output_data = xr.apply_ufunc(obtain_VT_data,                                       # The function that should be executed
                                 data.thetao, grdROMS, grdMODEL,                       # The arguments the function needs
                                 input_core_dims=[[], [], [], []],                     # For each argument
                                 exclude_dims=set(("lat",)),                           # The dimensions that are allowed to change size. Must be set!
                                )


    
    return outdat

In [ ]:
data.thetao

In [ ]:
def test_ufunc(data, ):
    ''' This function should return an array or tuple of arrays. '''
    
    

In [ ]:
# First pass the function and then its arguments in correct order

output2 = xr.apply_ufunc(test_ufunc, output, 
                         
)

## New approach

In [ ]:
# Step 1: create function that loops over ROMS depth layers

In [91]:
def obtain_VT_data(input_data, zr, bathymetry, zs, Nroms, Ndata):
    ''' Function to obtain the vertical transformed data for a certain depth layer and grid location of the ROMS grid. A time series is returned. '''
    
    #print(f'Received data has shapes: input_data={input_data.shape}, zr={zr.shape}, bathymetry={bathymetry.shape}, zs={zs.shape}' )
    #print(f'Received data has values: bathymetry={bathymetry}' )
    
    outdat = []
    
    
    for kc in range(int(Nroms)): # Loop over ROMS depth layers (30)
        
    
        # Case 1: ROMS is deeper than GLORYS. This part searches for deepest good value if ROMS depth is deeper than GLORYS. 
        # This means that if no value, or only fill_value, is available from GLORYS where ROMS is deepest, the closest value from GLORYS is found by looping upward in the water column.

        # Between GLORYS and ROMS, CASE 1 will never happen
        if zr[kc] < zs[Ndata - 1]:

            outdat.append(input_data[Ndata - 1])

            #We do not want to give the deepest depth a fill_value, but there are no fill_values



        # Case 2: ROMS depth layer is shallower than GLORYS depth layer. 

        elif zr[kc] > zs[0]:   

            outdat.append(input_data[0])


        else:

            for kT in range(int(Ndata) - 1): # Do loop between surface and bottom of GLORYS depth layers, - 1 because we also check for the next GLORYS layer each step

                # Case 3: ROMS depth layer is deeper than some GLORYS depth layer, but shallower than the next GLORYS layer which is below bottom 

                if (zr[kc] <= zs[kT]) & (-(bathymetry) > zs[kT + 1]):

                    outdat.append(input_data[kT])

                    #We do not want to give the deepest depth a fill_value, but there are no fill_values

                # For ORA check whether case 4 is necessary

                # Case 5: ROMS layer in between two GLORYS layers. This is the typical case for most layers.

                elif (zr[kc] <= zs[kT]) & (zr[kc] >= zs[kT + 1]) & (-(bathymetry) <= zs[kT + 1]):

                    rz2 = abs((zr[kc] - zs[kT + 1]) / (abs(zs[kT + 1]) - abs(zs[kT])))

                    rz1 = 1.0 - rz2
                    
                    res = (rz1 * input_data[kT+1] + rz2 * input_data[kT])
                    
                    outdat.append(res)

                    
    #print(f'Output has shape: {outdat.shape}')              
    return np.asarray(outdat)
                    
                    

In [55]:
# Step 2: call this function using apply_ufunc such that it is applied to each grid point and time step

In [96]:
# Obtain variables
bathymetry = grdROMS.h
zr = grdROMS.z_r
zs = grdMODEL.z_r
Nroms = int(grdROMS.nlevels)
Ndata = int(grdMODEL.nlevels)
xi_rho = int(grdROMS.xi_rho_)
eta_rho = int(grdROMS.eta_rho_)
fill=-10000

dat = data.copy()

# Change the name of 'depth' to 's_rho'
dat = dat.rename({'depth' : 's_rho'})
zs = zs.rename({'depth' : 's_rho'})

# Transpose dimensions
zr = zr.transpose('s_rho', 'xi_rho', 'eta_rho')
bathymetry = bathymetry.transpose('xi_rho', 'eta_rho')


# Change the arrangememnt of zr
zr = zr.sortby('s_rho', ascending = False)
    



In [94]:
# I'm certain about input_core_dims=[['s_rho'], ['s_rho'], [], ['s_rho'], [], []]

dtype('float32')

In [92]:
dat = dat.thetao.transpose('s_rho', 'time', 'xi_rho', 'eta_rho')


output_data = xr.apply_ufunc(obtain_VT_data,                                                     # The function that should be executed
                             dat, zr, bathymetry, zs, Nroms, Ndata,                              # The arguments the function needs
                             input_core_dims=[['s_rho'], ['s_rho'], [], ['s_rho'], [], []],      # The list of core dimensions on each input argument that should not be broadcast
                             exclude_dims=set(("s_rho",)), 
                             output_core_dims = [['s_rho']],
                             dask = 'parallelized',
                             output_dtypes = [dat.dtype],
                             vectorize = True)

In [95]:
output_data

<xarray.DataArray (time: 36, xi_rho: 122, eta_rho: 110, s_rho: 30)>
array([[[[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [17.89062286, 17.88740236, 17.88627184, ...,  2.1250533 ,
           2.0122013 ,  1.96759274],
         [16.97323362, 16.96831959, 16.96613158, ...,  2.26294332,
           2.09627078,  1.99896701],
         ...,
         [ 7.53158141,  7.53220939,  7.53266382, ...,  3.60903505,
           3.38432992,  3.18026581],
         [ 7.56514283,  7.56584131,  7.56596088, ...,  3.94346976,
           3.76374878,  3.57427491],
         [ 7.49281407,  7.49291278,  7.49353266, ...,  4.51035911,
           4.07559479,  4.00725412]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [17.69761822, 17.69346417, 17.69261812, ...,  2.08448927,
           1.99165069,  1.96041005],
         [16.82021245, 16.81378847, 16.81138849, ...,  2.13199659,
           2.01292514,         nan],
...
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]],

        [[        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         ...,
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan],
         [        nan,         nan,         nan, ...,         nan,
                  nan,         nan]]]])
Coordinates:
  * time     (time) datetime64[ns] 1993-01-16T12:00:00 ... 1995-12-16T12:00:00
  * eta_rho  (eta_rho) int64 0 1 2 3 4 5 6 7 ... 102 103 104 105 106 107 108 109
  * xi_rho   (xi_rho) int64 0 1 2 3 4 5 6 7 ... 114 115 116 117 118 119 120 121
Dimensions without coordinates: s_rho

In [97]:
dat = dat.so.transpose('s_rho', 'time', 'xi_rho', 'eta_rho')

output_data = xr.apply_ufunc(obtain_VT_data,                                                     # The function that should be executed
                             dat, zr, bathymetry, zs, Nroms, Ndata,                              # The arguments the function needs
                             input_core_dims=[['s_rho'], ['s_rho'], [], ['s_rho'], [], []],      # The list of core dimensions on each input argument that should not be broadcast
                             exclude_dims=set(("s_rho",)), 
                             output_core_dims = [['s_rho']],
                             dask = 'parallelized',
                             output_dtypes = [dat.dtype],
                             vectorize = True)

In [98]:
output_data

<xarray.DataArray (time: 36, xi_rho: 122, eta_rho: 110, s_rho: 30)>
array([[[[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [36.50412 , 36.50449 , 36.50449 , ..., 34.908268, 34.898518,
          34.89548 ],
         [36.334267, 36.334267, 36.334267, ..., 34.92123 , 34.906364,
          34.898205],
         ...,
         [35.136604, 35.136852, 35.136852, ..., 34.92383 , 34.94335 ,
          34.95911 ],
         [35.14266 , 35.14266 , 35.14266 , ..., 34.9365  , 34.945904,
          34.950375],
         [35.13934 , 35.13934 , 35.13934 , ..., 34.9514  , 34.93483 ,
          34.932224]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [36.48152 , 36.482418, 36.483006, ..., 34.904522, 34.895718,
          34.894863],
         [36.34141 , 36.34182 , 36.34228 , ..., 34.9089  , 34.89758 ,
                nan],
...
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]],

        [[      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         ...,
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan],
         [      nan,       nan,       nan, ...,       nan,       nan,
                nan]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1993-01-16T12:00:00 ... 1995-12-16T12:00:00
  * eta_rho  (eta_rho) int64 0 1 2 3 4 5 6 7 ... 102 103 104 105 106 107 108 109
  * xi_rho   (xi_rho) int64 0 1 2 3 4 5 6 7 ... 114 115 116 117 118 119 120 121
Dimensions without coordinates: s_rho

## model2roms.py STdata = vertical_interpolation(myvar, array1, array1, grdROMS, grdMODEL)

def vertical_interpolation(myvar, array1, array2, grdROMS, grdMODEL):
    outINDEX_ST = (grdROMS.nlevels, grdROMS.eta_rho, grdROMS.xi_rho)
    outINDEX_U = (grdROMS.nlevels, grdROMS.eta_u, grdROMS.xi_u)
    outINDEX_UBAR = (grdROMS.eta_u, grdROMS.xi_u)
    outINDEX_V = (grdROMS.nlevels, grdROMS.eta_v, grdROMS.xi_v)
    outINDEX_VBAR = (grdROMS.eta_v, grdROMS.xi_v)

    if myvar in ['salinity', 'temperature', 'O3_c', 'O3_TA', 'N1_p', 'N3_n', 'N5_s', 'O2_o']:
        logging.info(
            'Start vertical interpolation for {} (dimensions={} x {})'.format(myvar, grdROMS.xi_rho, grdROMS.eta_rho))
        outdata = np.empty((outINDEX_ST), dtype=np.float, order='F')

        outdata = interp.interpolation.dovertinter(np.asarray(outdata, order='F'),
                                                   np.asarray(array1, order='F'),
                                                   np.asarray(grdROMS.h, order='F'),
                                                   np.asarray(grdROMS.z_r, order='F'),
                                                   np.asarray(grdMODEL.z_r, order='F'),
                                                   int(grdROMS.nlevels),
                                                   int(grdMODEL.nlevels),
                                                   int(grdROMS.xi_rho),
                                                   int(grdROMS.eta_rho),
                                                   int(grdROMS.xi_rho),
                                                   int(grdROMS.eta_rho))

        outdata = np.ma.masked_where(abs(outdata) > 1000, outdata)
        # The BCG has to be capped at 0
        if myvar in ['O3_c', 'O3_TA', 'N1_p', 'N3_p', 'N3_n', 'N5_s', 'O2_o']:
            outdata = np.ma.masked_where(abs(outdata) < 0, outdata)
        # import plotData
        # for k in range(grdROMS.nlevels):
        #     plotData.contourMap(grdROMS, grdROMS.lon_rho, grdROMS.lat_rho, np.squeeze(outdata[k,:,:]),k, varname)

        return outdata

    if myvar == 'vvel':
        logging.info('Start vertical interpolation for uvel (dimensions={} x {})'.format(grdROMS.xi_u, grdROMS.eta_u))
        outdataU = np.zeros((outINDEX_U), dtype=np.float)
        outdataUBAR = np.zeros((outINDEX_UBAR), dtype=np.float)

        outdataU = interp.interpolation.dovertinter(np.asarray(outdataU, order='F'),
                                                    np.asarray(array1, order='F'),
                                                    np.asarray(grdROMS.h, order='F'),
                                                    np.asarray(grdROMS.z_r, order='F'),
                                                    np.asarray(grdMODEL.z_r, order='F'),
                                                    int(grdROMS.nlevels),
                                                    int(grdMODEL.nlevels),
                                                    int(grdROMS.xi_u),
                                                    int(grdROMS.eta_u),
                                                    int(grdROMS.xi_rho),
                                                    int(grdROMS.eta_rho))

        outdataU = np.ma.masked_where(abs(outdataU) > 1000, outdataU)

        logging.info('Start vertical interpolation for vvel (dimensions={} x {})'.format(grdROMS.xi_v, grdROMS.eta_v))
        outdataV = np.zeros((outINDEX_V), dtype=np.float)
        outdataVBAR = np.zeros((outINDEX_VBAR), dtype=np.float)

        outdataV = interp.interpolation.dovertinter(np.asarray(outdataV, order='F'),
                                                    np.asarray(array2, order='F'),
                                                    np.asarray(grdROMS.h, order='F'),
                                                    np.asarray(grdROMS.z_r, order='F'),
                                                    np.asarray(grdMODEL.z_r, order='F'),
                                                    int(grdROMS.nlevels),
                                                    int(grdMODEL.nlevels),
                                                    int(grdROMS.xi_v),
                                                    int(grdROMS.eta_v),
                                                    int(grdROMS.xi_rho),
                                                    int(grdROMS.eta_rho))

        outdataV = np.ma.masked_where(abs(outdataV) > 1000, outdataV)

        z_wu = np.zeros((grdROMS.nlevels + 1, grdROMS.eta_u, grdROMS.xi_u), dtype=np.float)
        z_wv = np.zeros((grdROMS.nlevels + 1, grdROMS.eta_v, grdROMS.xi_v), dtype=np.float)

        outdataUBAR = barotropic.velocity.ubar(np.asarray(outdataU, order='F'),
                                               np.asarray(outdataUBAR, order='F'),
                                               np.asarray(grdROMS.z_w, order='F'),
                                               np.asarray(z_wu, order='F'),
                                               grdROMS.nlevels,
                                               grdROMS.xi_u,
                                               grdROMS.eta_u,
                                               grdROMS.xi_rho,
                                               grdROMS.eta_rho)
        outdataUBAR = np.ma.masked_where(abs(outdataUBAR) > 1000, outdataUBAR)

        # plotData.contourMap(grdROMS, grdROMS.lon_rho, grdROMS.lat_rho, outdataUBAR,1, "ubar")

        outdataVBAR = barotropic.velocity.vbar(np.asarray(outdataV, order='F'),
                                               np.asarray(outdataVBAR, order='F'),
                                               np.asarray(grdROMS.z_w, order='F'),
                                               np.asarray(z_wv, order='F'),
                                               grdROMS.nlevels,
                                               grdROMS.xi_v,
                                               grdROMS.eta_v,
                                               grdROMS.xi_rho,
                                               grdROMS.eta_rho)

        # plotData.contourMap(grdROMS, grdROMS.lon_rho, grdROMS.lat_rho, outdataVBAR,1, "vbar")
        outdataVBAR = np.ma.masked_where(abs(outdataVBAR) > 1000, outdataVBAR)

        return outdataU, outdataV, outdataUBAR, outdataVBAR

interpolation.f90


Module interpolation
        implicit none
                
        contains
  
            subroutine doVertInter(outdat,dat,bathymetry,zr,zs,Nroms,Nsoda,II,JJ,xi_rho,eta_rho)
            
            ! ----------------------------------
            ! Program : doVertInter
            !
            ! This routine interpolates from z-levels to sigma levels using linear interpolation.
            !
            ! The index values in python goes from 0 toN while in Fortran they run from 1 to N+1. This is important to
            ! remember if one wants to compare input index wtih output index in fortran and python.
            !
            ! This routine assumes that the two depth matrixes zr (ROMS) and zs (SODA) are arranged from shallowest
            ! (index=1) to deepest (index=N+1). The depth matrizes must also be negative (if positive, reverse all
            ! comparison signs (e.g. LT, GT) in the program or multiply with minus 1). The input data are arranged with
            ! deepest values at highest index (N+1 e.g. dat(N+1)==bottom, dat(1)==surface). This is done so because
            ! it is the way SODA data are organized (bottom at highest index). However, ROMS output files are organized vice versa, so
            ! to accomodate that the output values are stored according to the ROMS structure. Highest index (N+1) equals surface,
            ! while lowest index equals bottom (index=1)(see how outdat(kc,jc,ic) is used opposite of the loop over kc).
            !
            ! Trond Kristiansen, December 2008, January, and March 2009
            ! Rutgers University, NJ.
            ! -------------------------------------------------------------------------------------------------------
            
            
            
            REAL(4) rz2, rz1, fill
            integer eta_rho, xi_rho, II, JJ, ic, jc, kc, kT, kkT, Nsoda, Nroms
            REAL(4), dimension(Nsoda,JJ,II) :: dat
            REAL(4), dimension(eta_rho,xi_rho) :: bathymetry
            REAL(4), dimension(Nroms,JJ,II) :: outdat
            REAL(4), dimension(Nsoda) ::  zs
            REAL(4), dimension(Nroms,eta_rho,xi_rho) :: zr

!f2py intent(in,out,overwrite) outdat       
!f2py intent(in,overwrite) dat, bathymetry, zr, zs
!f2py intent(in,overwrite) Nroms, Nsoda, JJ, II, xi_rho, eta_rho
!f2py intent(hide) ic,jc,kc,kT,rz1,rz2, kkT
            fill=-10000
            do jc=1,JJ
              do ic=1,II
                  do kc=1,Nroms
                      ! CASE 1: ROMS deeper than SODA. This part searches for deepest good value if ROMS depth is deeper
                      ! than SODA. This means that if no value, or only fill_value, is available from SODA where ROMS is
                      ! deepest, the closest value from SODA is found by looping upward in the water column.
                      IF (zr(kc,jc,ic) .LT. zs(Nsoda)) THEN
                          outdat(kc,jc,ic)=dat(Nsoda,jc,ic)
                          if (MAXVAL(dat(:,jc,ic)) .GT. fill) then
                            if (dat(Nsoda,jc,ic) .LT. fill) then
                              !print*,'Inside dovert and finding deepest depth with good values. current',dat(Nsoda,jc,ic)
                              DO kT=1,Nsoda
                                if (dat(Nsoda-kT,jc,ic) .GT. fill) then
                                    print*,'working on depth',kT,'with value',dat(kT,jc,ic)
                                    outdat(kc,jc,ic)=dat(Nsoda-kT,jc,ic)
                                    print*,'Able to find good value at depth ', Nsoda-kT
                                    exit
                                end if
                              end do
                             end if
                            end if
                          !print*,zr(kc,jc,ic),zs(Nsoda),dat(Nsoda,jc,ic),jc,ic,'case 1'
                    
                      ! CASE 2: ROMS shallower than SODA
                      ELSE IF (zr(kc,jc,ic) .GT. zs(1)) THEN
                          outdat(kc,jc,ic)=dat(1,jc,ic)
                     
                      ELSE
                          ! DO LOOP BETWEEN SURFACE AND BOTTOM
                          DO kT=1,Nsoda
                              ! CASE 3: ROMS deeper than SODA for one layer, but shallower than next SODA layer (bottom in between)
                              ! Deeper than some SODA depth layer, but shallower than next layer which is below bottom
                              IF (zr(kc,jc,ic) .LE. zs(kT) .AND.               &
                                -(bathymetry(jc,ic)) .GT. zs(kT+1)) THEN
                                outdat(kc,jc,ic)=dat(kT,jc,ic)
                                
                                ! We do not want to give the deepest depth a fill_value, so we find
                                ! the closest value to deepest depth.
                                if (MAXVAL(dat(:,jc,ic)) .GT. fill) then
                               
                                    if (dat(kT,jc,ic) .LE. fill) then
                                       !print*,'case3:Need to find better value for depth ',kT,'which has value ',dat(kT,jc,ic)
                                        DO kkT=1,Nsoda
                                            if (dat(kT-kkT,jc,ic) .GT. fill) then
                                                 outdat(kc,jc,ic)=dat(kT-kkT,jc,ic)
                            
                                                exit
                                            end if
                                        end do
                                    end if
                                end if
                                
                                ! CASE 4: Special case where ROMS layers are much deeper than in SODA
                                ELSE IF (zr(kc,jc,ic) .LE. zs(kT) .AND. dat(kT,jc,ic) .GT. fill &
                                .AND. dat(kT+1,jc,ic) .LE. fill) THEN
                                outdat(kc,jc,ic)=dat(kT,jc,ic)
                              
                              
                              ! CASE 5: ROMS layer in between two SODA layers
                              ! This is the typical case for most layers
                              ELSE IF ( (zr(kc,jc,ic) .LE. zs(kT)) .AND.       &
                              (zr(kc,jc,ic) .GE. zs(kT+1)) .AND.               &
                              (-bathymetry(jc,ic) .LE. zs(kT+1)) ) THEN
                              
                                 rz2 = abs((zr(kc,jc,ic)-zs(kT+1))/            &
                                 (abs(zs(kT+1))-abs(zs(kT))))
                                 
                                 rz1 = 1.0-rz2
        
                                 outdat(kc,jc,ic) = (rz1*dat(kT+1,jc,ic) &
                                 + rz2*dat(kT,jc,ic))
                                
                                if (MAXVAL(dat(:,jc,ic)) .GT. fill) then
                               
                                    if (dat(kT,jc,ic) .LE. fill .OR. dat(kT+1,jc,ic) .LE. fill) then
                                       !print*,'case4:Need to find better value for depth ',kT,kT+1,'which has &
                                   !    values ',dat(kT,jc,ic),dat(kT+1,jc,ic)
                                        DO kkT=1,Nsoda
                                            if (dat(kT-kkT,jc,ic) .GT. fill .and. dat(kT-kkT+1,jc,ic) .GT. fill  ) then
                                                 !print*,'CASE 4: Found good value at depth',kT-kkT,kt-kkT+1
                                                 !print*,'with values',dat(kT-kkT,jc,ic), dat(kt-kkT+1,jc,ic)
                                                 outdat(kc,jc,ic) = (rz1*dat(kT+1-kkT,jc,ic) &
                                                 + rz2*dat(kT-kkT,jc,ic))
                            
                                                exit
                                            end if
                                        END DO
                                    end if
                                 end if
                                
                                 exit
                                 
                              END IF
                          ! DO LOOP BETWEEN SURFACE AND BOTTOM: CASE 3,4,5
                          END DO
                      ! TEST ALL CASES IF LOOP: CASE 1,2,3,4,5
                      END IF
                     
                  end do
              end do
            end do
            
        
      end subroutine doVertInter

In [ ]:
# Indices of output data, however my output data includes Time
index_time = data.time.size


outINDEX_ST = (int(grdROMS.nlevels), index_time, int(grdROMS.eta_rho_), int(grdROMS.xi_rho_))

outdata = np.empty((outINDEX_ST))

### Check whether input data is processed correct before VT


The input data for the VT consists of zr, zs and the GLORYS reanalysis data. According to the model2roms code:

- This routine assumes that the two depth matrixes zr (ROMS) and zs (SODA) are arranged from shallowest (index=1) to deepest (index=N+1). 
  The depth matrizes must also be negative (if positive, reverse all comparison signs (e.g. LT, GT) in the program or multiply with minus 1). 

- The input data are arranged with deepest values at highest index (N+1 e.g. dat(N+1)==bottom, dat(1)==surface). This is done so because it is the way SODA data are organized (bottom at highest index). 

- However, ROMS output files are organized vice versa, so to accomodate that the output values are stored according to the ROMS structure. 
  Highest index (N+1) equals surface, while lowest index equals bottom (index=1)(see how outdat(kc,jc,ic) is used opposite of the loop over kc).



Deeper values are connected to lower numbers for s_rho. However, for the variable 'depth', higher numbers are connected to deeper values.

In [ ]:
zr.isel(eta_rho=0, xi_rho=0)

In [ ]:
zr.s_rho

In [ ]:
zs

In [ ]:
zs.depth

- zs and zr are negative --> correct

- zr is arranged from deepest to shallowest --> not correct

- zs is arranged from shallowest to deepest --> correct

- The Glorys input data is arranged from shallowest to deepest --> correct



In [ ]:
# Change the arrangememnt of zr

zr = zr.sortby('s_rho', ascending = False)

### Perform Vertical Transformation

In [ ]:
for tc in range(dat.time.size): # Loop over time

    

In [ ]:
zr.isel(s_rho = -1).plot()

In [ ]:
new = -1*bathymetry
new.plot()

In [ ]:
zr.min()

In [ ]:
zs.plot()

In [ ]:
zs[0]

In [ ]:
dat.thetao.min()

In [ ]:
# ADD A TQDM aROUND JJ for loop

In [ ]:
for jc in range(JJ): # Loop over eta_rho direction (110)

    for ic in range(II): # Loop over xi_rho direction (122)

        for kc in range(int(Nroms)): # Loop over ROMS depth layers (30)
                
            # Case 1: ROMS is deeper than GLORYS. This part searches for deepest good value if ROMS depth is deeper than GLORYS. 
            # This means that if no value, or only fill_value, is available from GLORYS where ROMS is deepest, the closest value from GLORYS is found by looping upward in the water column.
            
            # Between GLORYS and ROMS, CASE 1 will never happen
            
            if zr[kc, jc, ic] < zs[Ndata - 1]:

                outdat[kc, :, jc, ic] = dat.thetao[Ndata - 1, :, jc, ic]
                
                #We do not want to give the deepest depth a fill_value, but there are no fill_values



        # Case 2: ROMS depth layer is shallower than GLORYS depth layer. 
                                                  
            elif zr[kc, jc, ic] > zs[0]:    # zs[0] = -0.494025, only the case for kc = 0
                                                  
                outdat[kc, :, jc, ic] = dat.thetao[0, :, jc, ic]
        
                                                
            else:
                    
                for kT in range(int(Ndata) - 1): # Do loop between surface and bottom of GLORYS depth layers, - 1 because we also check for the next GLORYS layer each step
                      
                    # Case 3: ROMS depth layer is deeper than some GLORYS depth layer, but shallower than the next GLORYS layer which is below bottom 
                                                  
                    if (zr[kc, jc, ic] <= zs[kT]) & (-(bathymetry[jc, ic]) > zs[kT+1]):
                            
                        outdat[kc, :, jc, ic] = dat.thetao[kT, :, jc, ic]
                        
                        #We do not want to give the deepest depth a fill_value, but there are no fill_values
                         
                            
                            
                    # Case 5: ROMS layer in between two GLORYS layers. This is the typical case for most layers.
                        
                    elif (zr[kc, jc, ic] <= zs[kT]) & (zr[kc, jc, ic] >= zs[kT+1]) & (-bathymetry[jc, ic] <= zs[kT+1]):
                            
                        rz2 = abs((zr[kc, jc, ic] - zs[kT+1]) / (abs(zs[kT+1]) - abs(zs[kT])))
                              
                        rz1 = 1.0-rz2
                            
                        outdat[kc, :, jc, ic] = (rz1 * dat.thetao[kT+1, :, jc, ic] + rz2 * dat.thetao[kT, :, jc, ic])
                            
                        

In [ ]:
outdat

In [ ]:
for jc in range(JJ): # Loop over eta_rho direction (110)

    for ic in range(II): # Loop over xi_rho direction (122)

        for kc in range(int(Nroms)): # Loop over ROMS depth layers (30)
                
            # Case 1: ROMS is deeper than GLORYS. This part searches for deepest good value if ROMS depth is deeper than GLORYS. 
            # This means that if no value, or only fill_value, is available from GLORYS where ROMS is deepest, the closest value from GLORYS is found by looping upward in the water column.
            
            # Between GLORYS and ROMS, CASE 1 will never happen
            
            if zr[kc, jc, ic] < zs[Ndata - 1]:

                outdat[kc, :, jc, ic] = dat.thetao[Ndata - 1, :, jc, ic]
                
                for tc in range(dat.time.size):
                    
                    if dat.thetao[:, tc, jc, ic].max() > fill:  # Always the case

                        if dat.thetao[Ndata - 1, tc, jc, ic] < fill: # These data values are always NaN. So this is never the case


                            for kT in range(int(Ndata)):

                                if dat.thetao[Ndata - 1 - kT, tc, jc, ic] > fill:


                                    outdat[kc, tc, jc, ic] = dat.thetao[Ndata - 1 - kT, tc, jc, ic]



        # Case 2: ROMS depth layer is shallower than GLORYS depth layer. 
                                                  
            elif zr[kc, jc, ic] > zs[0]:    # zs[0] = -0.494025, only the case for kc = 0
                                                  
                outdat[kc, :, jc, ic] = dat.thetao[0, :, jc, ic]
        
                                                
            else:
                    
                for kT in range(int(Ndata) - 1): # Do loop between surface and bottom of GLORYS depth layers, - 1 because we also check for the next GLORYS layer each step
                      
                    # Case 3: ROMS depth layer is deeper than some GLORYS depth layer, but shallower than the next GLORYS layer which is below bottom 
                                                  
                    if (zr[kc, jc, ic] <= zs[kT]) & (-(bathymetry[jc, ic]) > zs[kT+1]):
                            
                        outdat[kc, :, jc, ic] = dat.thetao[kT, :, jc, ic]
                        
                        #We do not want to give the deepest depth a fill_value, so we find the closest value to deepest depth.
                            
                        for tc in range(dat.time.size):
                            
                            if dat.thetao[:, tc, jc, ic].max() > fill:

                                if dat.thetao[kT, tc, jc, ic] <= fill:


                                    for kkt in range(int(Ndata)):

                                        if dat.thetao[kT-kkT, tc, jc, ic] > fill:

                                            outdat[kc, tc, jc, ic] = dat.thetao[kT-kkT, tc, jc, ic]
                                                
                                                
                            
                    # Case 4 (special case) where ROMS layers are much deeper than GLORYS layers 
                
                    elif (zr[kc, jc, ic] <= zs[kT]) & (dat.thetao[kT, tc, jc, ic] > fill) & (dat.thetao[kT+1, tc, jc, ic] <= fill):
                            
                        outdat[kc, :, jc, ic] = dat.thetao[kT, :, jc, ic]
                            
                            
                    # Case 5: ROMS layer in between two GLORYS layers. This is the typical case for most layers.
                        
                    elif (zr[kc, jc, ic] <= zs[kT]) & (zr[kc, jc, ic] >= zs[kT+1]) & (-bathymetry[jc, ic] <= zs[kT+1]):
                            
                        rz2 = abs((zr[kc, jc, ic] - zs[kT+1]) / (abs(zs[kT+1]) - abs(zs[kT])))
                              
                        rz1 = 1.0-rz2
                            
                        outdat[kc, :, jc, ic] = (rz1 * dat.thetao[kT+1, tc, jc, ic] + rz2 * dat.thetao[kT, tc, jc, ic])
                            
                        # MAYBE PERFORM TIME LOOP HERE
                        if dat.thetao[:, tc, jc, ic].max() > fill:
                                
                            if dat.thetao[kT, tc, jc, ic] <= fill or dat.thetao[kT + 1, tc, jc, ic] <= fill:
                                    
                                        
                                for kkT in range(Ndata):
                                            
                                    if (dat.thetao[kT-kkT, tc, jc, ic] > fill) & (dat.thetao[kT-kkT+1, tc, jc, ic] > fill):
                                                
                                                
                                        outdat[kc, tc, jc, ic] = rz1 * dat.thetao[kT+1-kkT, tc, jc, ic] + rz2 * dat.thetao[kT-kkT, tc, jc, ic]
                                
                                
                                

In [ ]:




#Obtain s_rho and z_rho

hc = np.asarray(grid.hc)
h = np.asarray(grid.h)
theta_s = np.asarray(grid.theta_s)
theta_b = np.asarray(grid.theta_b)

s_rho = np.asarray(grid.s_rho)
nlevels = len(s_rho)


zeta = np.zeros(h.shape)

# Obtain s_rho
N = int(nlevels)
lev = np.arange(1, N + 1, 1)

c1 = 1.0
c2 = 2.0
p5 = 0.5
    
ds = 1.0 / N

s_rho = - c1 + (lev - p5) * ds

# Obtain Cs_r
if (theta_s > 0):
    Csur = (c1 - np.cosh(theta_s * s_rho)) / (np.cosh(theta_s) - c1)
        
else:
    Csur = -s_rho**2
        
if (theta_b > 0):
    Cbot = (np.exp(theta_b * Csur) - c1 ) / (c1 - np.exp(-theta_b))
    Cs_r = Cbot
else:
    Cs_r = Csur     

    
z_r = np.empty((N,)+h.shape, 'd')

for k in range(N):
    z0 = (hc * s_rho[k] + h * Cs_r[k]) / (hc + h)

    z_r[k, :] = zeta + (zeta + h) * z0
    
    
    
# IMPROVE BY USING XARRAY

In [ ]:
# Create the dataset grdROMS

write_clim = True
write_bry = True
write_init = True
write_stations = False
lonname = 'lon_rho'
latname = 'lat_rho'
inittime = 0               # Set initTime to 1 if you dont want the first time-step to be the initial field (no ubar and vbar if time=0)
ocean_time = 0
NT = 2
tracer = NT
time = 0

reftime = 0
grdtype = 'regular'
mask_rho = grid.mask_rho
lon_rho = grid.lon_rho
lat_rho = grid.lat_rho
h = grid.h

masked_h = np.where(h > 0, h, h.max())

hmin = masked_h.min()

vtransform = 2
vstretching = 4

s_rho = grid.s_rho
nlevels = len(s_rho)

theta_s = grid.theta_s
theta_b = grid.theta_b
tcline = grid.Tcline
hc = grid.hc

zeta = np.zeros(h.shape)

lon_u = grid.lon_u
lat_u = grid.lat_u

mask_u = grid.mask_u

lon_vert = grid.lon_vert
lat_vert = grid.lat_vert

x_rho = grid.x_rho
y_rho = grid.y_rho

x_u = grid.x_u
y_u = grid.y_u

x_v = grid.x_v
y_v = grid.y_v

x_psi = grid.x_psi
y_psi = grid.y_psi

x_vert = grid.x_vert
y_vert = grid.y_vert

xl = grid.xl
el = grid.el

dmde = grid.dmde
dndx = grid.dndx

lon_v = grid.lon_v
lat_v = grid.lat_v
mask_v = grid.mask_v

lon_psi = grid.lon_psi
lat_psi = grid.lat_psi
mask_psi = grid.mask_psi

angle = grid.angle

pm = grid.pm
invpm = 1.0 / np.asarray(pm)

pn = grid.pn
invpn = 1.0 / np.asarray(pn)

Lp = len(lat_rho[1,:])
Mp = len(lat_rho[:,1])

fillval = -9.99e33

eta_rho = Mp
eta_u = Mp
eta_v = Mp - 1
eta_psi = Mp - 1

xi_rho = Lp
xi_u = Lp - 1
xi_v = Lp
xi_psi = Lp - 1



def s_coordinate_4():
    
    
    h_arr = np.asarray(h)
    hmin = h_arr.min()
    N = int(nlevels)
    Np = N + 1


    Vtrans = 4

    c1 = 1.0
    c2 = 2.0
    p5 = 0.5
    
    # Obtain s_rho
    lev = np.arange(1, N + 1, 1)
    ds = 1.0 / N
    s_rho = - c1 + (lev - p5) * ds

    
    # Obtain s_w
    lev = np.arange(0, Np, 1)
    ds = 1.0 / (Np - 1)
    s_w = - c1 + lev * ds
    
    
    # Obtain Cs_r
    if (theta_s > 0):
        Csur = (c1 - np.cosh(theta_s * s_rho)) / (np.cosh(theta_s) - c1)
        
    else:
        Csur = -s_rho**2
        
    if (theta_b > 0):
        Cbot = (np.exp(theta_b * Csur) - c1 ) / (c1 - np.exp(-theta_b))
        Cs_r = Cbot
    else:
        Cs_r = Csur     
    
    
    
    # Obtain Cs_w
    if (theta_s > 0):
        Csur = (c1 - np.cosh(theta_s * s_w)) / (np.cosh(theta_s) - c1)
        
    else:
        Csur = -s_w**2
    if (theta_b > 0):
        Cbot = (np.exp(theta_b * Csur) - c1 ) / ( c1 - np.exp(-theta_b) )
        
        Cs_w = Cbot
        
    else:
        Cs_w = Csur
            
            
            
            
    return s_rho, s_w, Cs_r, Cs_w


s_rho, s_w, Cs_r, Cs_w = s_coordinate_4()
  
    
    
def z_r():
    
    # Make sure there is a time dimension
    zeta = zeta[np.newaxis, :]
    
    time = zeta.shape[0]
    z_r = np.empty((ti, self.N) + self.h.shape, 'd')
    
    for n in range(time):
        for k in range(N):
            z0 = hc * s_rho[k] + h * Cs_r[k] / (hc + h)
            z_r[n,k,:] = zeta[n,:] + (zeta[n,:] + h) * z0
    
    return z_r




def z_w():
    
    z_w = 'hoi'
    
    return z_w()
    
            
            
z_r = z_r(h, hc, N, s_rho, Cs_r, zeta, Vtrans)
z_w = z_w(h, hc, Np, s_w, Cs_w, zeta, Vtrans)







#vgrid = s_coordinate_4()


# Also ESMF grid is added but probably not needed for VT


In [ ]:
s_rho.shape

In [ ]:
h_arr = np.asarray(h)
hmin = h_arr.min()
N = int(nlevels)
Np = N + 1


Vtrans = 4

c1 = 1.0
c2 = 2.0
p5 = 0.5
    
# Obtain s_rho
lev = np.arange(1, N + 1, 1)
ds = 1.0 / N
s_rho = - c1 + (lev - p5) * ds

    
# Obtain s_w
lev = np.arange(0, Np, 1)
ds = 1.0 / (Np - 1)
s_w = - c1 + lev * ds
    
    
# Obtain Cs_r
if (theta_s > 0):
    Csur = (c1 - np.cosh(theta_s * s_rho)) / (np.cosh(theta_s) - c1)
        
else:
    Csur = -s_rho**2

In [ ]:
zeta.shape

In [ ]:
zeta = zeta[np.newaxis, :]
zeta.shape

In [ ]:
grid.mask_psi

In [ ]:
grid.angle